<a href="https://colab.research.google.com/github/jmoro0408/sloped_pipe_volume/blob/main/many_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go


In [2]:
# initial values
initial_grade_percent = 0.2 #inital grade value
maximum_grade_value = 2 #maximum grade (%) to plot 
no_grades = 10 #grades will range equally between inital grade value and maximum grade value by this many points
pipe_length = 20
no_points_to_plot = 100
pipe_chainage = np.linspace(0, 200, no_points_to_plot).tolist()

pipe_diameter = 2 
pipe_invert_start = 0 #pipe invert at pipe end
lower_liquid_level = 1 #liquid level at pipe end



In [3]:
def calculate_tilt_angle(initial_grade_percent):
  tilt_angle = np.arctan(initial_grade_percent / 100)
  tilt_angle = np.degrees(tilt_angle)
  return tilt_angle

def calculate_pipe_upper_invert(pipe_invert_elev,tilt_angle,pipe_length):
  pipe_upper_invert = np.tan(np.radians(tilt_angle)) * pipe_length + pipe_invert_elev
  return pipe_upper_invert

def calculate_upper_liquid_level(lower_liquid_level, tilt_angle, pipe_length):
  upper_liquid_level = lower_liquid_level - (pipe_length *np.tan(np.radians(tilt_angle)))
  return upper_liquid_level


In [4]:
tilt_angle = calculate_tilt_angle(initial_grade_percent)
upper_pipe_invert = calculate_pipe_upper_invert(pipe_invert_start,tilt_angle,pipe_length)
upper_liquid_level = calculate_upper_liquid_level(lower_liquid_level, tilt_angle, pipe_length)

In [5]:
#create x points along the chainage
pipe_invert_points = np.linspace(pipe_invert_start, upper_pipe_invert, no_points_to_plot)

# multiple plots

Need to create plot lines for each grade percentage in slider list

In [6]:
initial_grade_percent_list = np.linspace(0, maximum_grade_value, no_grades) #this will be the slider eventually

In [7]:
#Building a dictionary that holdes the grade and every respective pipe invert y coordinate 
pipe_invert_dict = {}
for grade in initial_grade_percent_list:
  _tilt_angle = calculate_tilt_angle(grade)
  _upper_pipe_invert = calculate_pipe_upper_invert(pipe_invert_start, _tilt_angle,pipe_length)
  _pipe_invert_points = np.linspace(pipe_invert_start, _upper_pipe_invert, no_points_to_plot)
  _pipe_invert_points = _pipe_invert_points.tolist()
  pipe_invert_dict.update({grade:_pipe_invert_points})


In [8]:
#Building a dictionary that holdes the grade and every respective pipe crown y coordinate 
# The pipe crown is just the invert + the pipe diameter, so this is easy to build from the invert dict
pipe_crown_dict = {}
for key, values in pipe_invert_dict.items():
  _crown_value = [value + pipe_diameter for value in values]
  pipe_crown_dict.update({key:_crown_value})


In [9]:
# Building a dict to hold the respective upper liquid levels at every grade
liquid_level_dict = {}

for grade in initial_grade_percent_list:
  _tilt_angle = calculate_tilt_angle(grade)
  _upper_liquid_level = calculate_upper_liquid_level(lower_liquid_level, _tilt_angle, pipe_length)
  _upper_liquid_level_points = np.linspace(lower_liquid_level,_upper_liquid_level,  no_points_to_plot)
  _upper_liquid_level_points = _upper_liquid_level_points.tolist()
  liquid_level_dict.update({grade:_upper_liquid_level_points})



In [22]:
"""this one is tricky. Trying to make sure any point where the liquid level is below the pipe invert is not plotted. 

I think there is a neater way to do this but I tried going directly to the go.Figure() object, and to the individual dicts but
got lost navigating the nested stucture. This way works for now. 
"""

pipe_invert_df = pd.DataFrame.from_dict(pipe_invert_dict) #create dataframes from both dicts
liquid_level_df = pd.DataFrame.from_dict(liquid_level_dict)

for column in pipe_invert_df:
  for index in pipe_invert_df.index:
    if liquid_level_df[column].iloc[index] < pipe_invert_df[column].iloc[index]: # if liquid level < pipe invert, replace with NaN
      liquid_level_df[column].iloc[index] = np.nan

liquid_level_dict = liquid_level_df.to_dict(orient = "list")

del pipe_invert_df, liquid_level_df #dont need these dataframes anymore

In [13]:
fig1 = go.Figure()
trace_list1 = []
trace_list2 = []
trace_list3 = []
for grade in pipe_invert_dict.keys():
  _tilt_angle = round(calculate_tilt_angle(grade),2)
  trace_list1.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=pipe_invert_dict[grade], 
                                name = "Pipe Invert",
                                showlegend=False, 
                                line=dict(color='gray', width=4)))
  
  trace_list2.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=pipe_crown_dict[grade], 
                                name = "Pipe Crown",
                                showlegend=False, 
                                line=dict(color='gray', width=4)))
  
  trace_list3.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=liquid_level_dict[grade], 
                                name = "Liquid Level", 
                                line=dict(color='royalblue', width=4)))

TypeError: ignored

In [12]:
#Building the slider
# in order to capture both the crown and invert moving together with a singler slider step we need to append both traces to a single list

num_steps = len(initial_grade_percent_list) #each step = a different grade
fig2 = go.Figure(data = trace_list1 + trace_list2+trace_list3)

#have to be really careful here. You want the slider to be aware of the ordering of traces, see https://community.plotly.com/t/multiple-traces-with-a-single-slider-in-plotly/16356

fig2.data[0].visible = True #this is the pipe invert data
fig2.data[num_steps].visible = True #pipe crown data
fig2.data[num_steps*2].visible = True #liquid level data


steps = []
for i in range(num_steps):
  step = dict(
      method="restyle", 
      args = [{"visible":[False] * len(fig2.data)}, 
              {"title":f"grade: {fig2.data[i]['name']}"}], 
              label=str(round(list(pipe_invert_dict.keys())[i],2)))
  
  step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible" - pipe invert
  step["args"][0]["visible"][i+num_steps] = True #pipe crown
  step["args"][0]["visible"][i+num_steps*2] = True #liquid level
  steps.append(step)

sliders = [dict( 
    active=0,
    currentvalue={"prefix": "Grade: ", "suffix": " %"},
    pad={"t": 50},
    steps=steps
)]


fig2.update_layout(
    template = "simple_white", 
    title="Liquid Level Along Pipe ",
    xaxis_title="Pipe Chainage (m)",
    yaxis_title="Elevation (mGeodetic)",
    sliders=sliders,
  #Legend info 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1),
)


fig2.update_yaxes(range=[min(min(pipe_invert_dict.values()))-1,max(max(pipe_crown_dict.values()))]) #have to lock the y axis, otherwise the axis just updates and the line doesnt "move"
fig2.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across")
fig2.update_yaxes(showspikes=True,spikecolor="green")

fig2.show()

